<a href="https://colab.research.google.com/github/Codewiz22/AES/blob/main/Data_Encryption_Standard_(DES)_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, modes
# ERROR FIX: Updated TripleDES import path to suppress deprecation warning
from cryptography.hazmat.decrepit.ciphers import algorithms as decrepit_algorithms
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

class DESCipher: # Keep the class name, but internally use TripleDES for compatibility
    """
    A professional-grade DES implementation for academic study.
    Standard: DES-CBC with PKCS7 Padding.

    DES is a historical symmetric-key algorithm that operates on
    64-bit blocks with a 64-bit key (of which only 56 bits are used).
    """
    def __init__(self, key=None):
        # DES requires an 8-byte (64-bit) key.
        # This is significantly shorter than AES, making it vulnerable to brute force.
        # ERROR FIX: Changed to 16 bytes for TripleDES, as DES might not be directly available.
        # TripleDES keys can be 16 (two-key) or 24 (three-key) bytes. Using 16 bytes here.
        self.key = key if key else os.urandom(16) # Changed key length to 16 for TripleDES
        self.backend = default_backend()

        if len(self.key) not in [16, 24]: # Adjusted validation for TripleDES
            raise ValueError("TripleDES key must be exactly 16 or 24 bytes long.")

    def encrypt(self, plaintext: str) -> dict:
        """
        Encrypts plaintext using DES-CBC.
        Returns a dictionary containing the ciphertext and the IV.
        """
        # DES uses a 64-bit block size, so the IV must be 8 bytes.
        iv = os.urandom(8)

        # Setup Padding (DES operates on 64-bit blocks)
        padder = padding.PKCS7(64).padder()
        padded_data = padder.update(plaintext.encode()) + padder.finalize()

        # Create Cipher object using DES algorithm in CBC mode
        # ERROR FIX: Changed algorithms.DES to algorithms.TripleDES
        # ERROR FIX: Using decrepit_algorithms.TripleDES to avoid deprecation warning
        cipher = Cipher(decrepit_algorithms.TripleDES(self.key), # Changed to decrepit_algorithms.TripleDES
                        modes.CBC(iv),
                        backend=self.backend)
        encryptor = cipher.encryptor()

        # Encrypt the padded data
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()

        return {
            "ciphertext": ciphertext,
            "iv": iv,
            "key_size_bits": len(self.key) * 8 # Still reflects actual key size
        }

    def decrypt(self, ciphertext: bytes, iv: bytes) -> str:
        """
        Decrypts ciphertext using DES-CBC.
        """
        # ERROR FIX: Changed algorithms.DES to algorithms.TripleDES
        # ERROR FIX: Using decrepit_algorithms.TripleDES to avoid deprecation warning
        cipher = Cipher(decrepit_algorithms.TripleDES(self.key), # Changed to decrepit_algorithms.TripleDES
                        modes.CBC(iv),
                        backend=self.backend)
        decryptor = cipher.decryptor()

        # Decrypt the ciphertext
        padded_data = decryptor.update(ciphertext) + decryptor.finalize()

        # Remove PKCS7 Padding
        unpadder = padding.PKCS7(64).unpadder()
        data = unpadder.update(padded_data) + unpadder.finalize()

        return data.decode('utf-8')

# --- Usage Example for Week 5 Presentation ---
if __name__ == "__main__":
    print("--- Data Encryption Standard (DES) Demo ---")

    # Initialize with an 8-byte key
    # ERROR FIX: Changed key generation to 16 bytes for TripleDES
    des_key = os.urandom(16) # Changed key length to 16 for TripleDES
    des = DESCipher(des_key)

    message = "DES is a legacy algorithm used for historical analysis."

    # Encrypt
    result = des.encrypt(message)
    print(f"Algorithm:       TripleDES-CBC (DES not found)") # Updated algorithm name
    # ERROR FIX: Updated effective key size for TripleDES (2-key TDES is 112 bits effective)
    print(f"Key Size:        {result['key_size_bits']} bits ({'112' if result['key_size_bits'] == 128 else '168'} bits effective for TripleDES)") # Adjusted effective bits
    print(f"Original:        {message}")
    print(f"Ciphertext(hex): {result['ciphertext'].hex()}")
    print(f"IV (hex):         {result['iv'].hex()}")

    # Decrypt
    decrypted = des.decrypt(result['ciphertext'], result['iv'])
    print(f"Decrypted:       {decrypted}")

    # Verification
    assert message == decrypted
    print("\nDES Encryption Cycle: Success (using TripleDES for compatibility).") # Updated success message


--- Data Encryption Standard (DES) Demo ---
Algorithm:       TripleDES-CBC (DES not found)
Key Size:        128 bits (112 bits effective for TripleDES)
Original:        DES is a legacy algorithm used for historical analysis.
Ciphertext(hex): 61622abbdec51491c68c31e1bd74372a97ef030b9b17f24fad458c1300a64622bb630b319dd02079d9556503eeb9115bbd2797eb9cd77407
IV (hex):         5572cb4eee610d74
Decrypted:       DES is a legacy algorithm used for historical analysis.

DES Encryption Cycle: Success (using TripleDES for compatibility).
